In [ ]:
!pip install mne
!pip install matplotlib

In [ ]:
import mne
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import freqz
from scipy.signal import butter, lfilter
from scipy.signal import spectrogram

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band',analog=True)
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

Remove unwanted EEG signals

In [ ]:
def cleanAndRaw(filePath):

    good_channels = ['EEG FP1-REF','EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF','EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF']
    good_channels = set(good_channels)

    eeg = mne.io.read_raw_edf(filePath)

    eeg = eeg.resample(100, npad='auto')  

    channels = eeg.info.ch_names

    eeg_19 = eeg.copy()
    for channel in channels:

        if channel not in good_channels:

            eeg_19.drop_channels([channel])

    raw_eeg = eeg_19.get_data()

    if raw_eeg.shape[0] == 19 and raw_eeg.shape[1] >= 36000:

        raw_eeg  = raw_eeg - raw_eeg.mean(axis=1,keepdims=True)

        fs = 100
        lowcut = 0.5
        highcut = 50

        t = np.arange(0,raw_eeg.shape[1],1)

        for i in range(19):
            raw_eeg[i] = butter_bandpass_filter(raw_eeg[i], lowcut, highcut, fs, order=5)

        raw_eeg = np.delete(raw_eeg,np.s_[:600], axis=1)    
        raw_eeg = np.delete(raw_eeg,np.s_[30000:], axis=1)    

        return raw_eeg

    else:

        return None

Make spectrograms and save them

In [ ]:
c = 0
for filename in os.listdir('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/class_1'):
    c += 1
print(c)

for filename in os.listdir('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/class_1'):
    raw = cleanAndRaw('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/class_1/'+filename)

    if raw is not None:

        images = []

        for i in range(19):    
            print(i,end=" ")
            spec,freq,t,im = plt.specgram(raw[i],Fs=100,NFFT=100,noverlap=50)
            plt.axis('off')
            figure = plt.gcf()
            figure.set_size_inches(12, 1)

            figure.canvas.draw()
            img = np.array(figure.canvas.buffer_rgba())
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGRA)

            b = figure.axes[0].get_window_extent()
            img = np.array(figure.canvas.buffer_rgba())
            img = img[int(b.y0):int(b.y1),int(b.x0):int(b.x1),:]
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGRA)
            
            images.append(img)

        base = cv2.vconcat(images)
        # imageAsArray = np.reshape(base,(1,2745576)) #! From (1026,669,4) -> (1,1026X669X4)

        # np.save('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/spectrograms/c2/image as ndarray/'+filename[:-4]+'.npz',imageAsArray)
        cv2.imwrite('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/spectrograms/c1/'+filename[:-4]+'.png',base)

    c -= 1
    print(c)


Save spectrograms as arrays

In [ ]:
c = 0
for filename in os.listdir('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/class_3'):
    c += 1
print(c)
for filename in os.listdir('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/class_3'):
    raw = cleanAndRaw('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/class_3/'+filename)
    print(c)
    c -= 1
    if raw is not None:
        if raw is not None:
            S = np.empty((0,51,599),np.float64)
            for i in range(19):
                spec,freq,t,im = plt.specgram(raw[i],Fs=100,NFFT=100,noverlap=50)
                S = np.append(S,np.array([spec]),axis=0)
            linearS = np.reshape(S,(1,580431)) #! From (19,51,599) -> (1,19X51X599)
            np.save('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/spectrograms/c3/spectrograms as csv/'+filename[:-4]+'.csv', linearS,delimiter=',')

Saving signals as ndarray


In [ ]:
c = 0
for filename in os.listdir('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/class_3'):
    c += 1
print(c)

for filename in os.listdir('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/class_3'):
    raw = cleanAndRaw('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/class_3/'+filename)
    if raw is not None:
        np.save('/home/roshan/Documents/7th_sem/MTP/Medication-from-EEG/spectrograms/c3/signal as ndarray/'+filename[:-4]+'.npz', raw)
    c -= 1
    print(c)